# Semantic Router: Hybrid Layer

The Hybrid Layer in the Semantic Router library can improve decision making performance particularly for niche use-cases that contain specific terminology, such as finance or medical. It helps us provide more importance to decision making based on the keywords contained in our utterances and user queries.

## Getting Started

We start by installing the library:

In [ ]:
!pip install -qU semantic-router==0.0.5

We start by defining a dictionary mapping decisions to example phrases that should trigger those decisions.

In [ ]:
import os

os.environ["COHERE_API_KEY"] = "BQBiUqqjDRsYl1QKKux4JsqKdDkjyInS5T3Z3eJP"

In [ ]:
from semantic_router.schema import Decision

politics = Decision(
    name="politics",
    utterances=[
        "isn't politics the best thing ever",
        "why don't you tell me about your political opinions",
        "don't you just love the president",
        "don't you just hate the president",
        "they're going to destroy this country!",
        "they will save the country!",
    ],
)

Let's define another for good measure:

In [ ]:
chitchat = Decision(
    name="chitchat",
    utterances=[
        "how's the weather today?",
        "how are things going?",
        "lovely weather today",
        "the weather is horrendous",
        "let's go to the chippy",
    ],
)

chitchat = Decision(
    name="chitchat",
    utterances=[
        "how's the weather today?",
        "how are things going?",
        "lovely weather today",
        "the weather is horrendous",
        "let's go to the chippy",
    ],
)

decisions = [politics, chitchat]

Now we initialize our embedding model:

In [ ]:
from semantic_router.encoders import CohereEncoder
from getpass import getpass

os.environ["COHERE_API_KEY"] = os.environ["COHERE_API_KEY"] or getpass(
    "Enter Cohere API Key: "
)

encoder = CohereEncoder()

Now we define the `DecisionLayer`. When called, the decision layer will consume text (a query) and output the category (`Decision`) it belongs to — to initialize a `DecisionLayer` we need our `encoder` model and a list of `decisions`.

In [ ]:
from semantic_router.layer import HybridDecisionLayer

dl = HybridDecisionLayer(encoder=encoder, decisions=decisions)

In [ ]:
dl("don't you love politics?")

In [ ]:
dl("how's the weather today?")

---